In [1]:
import numpy as np
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import *
from keras.layers import *
import os
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# defining other metrics:
def psnr(y_true,y_pred):
    return tf.image.psnr(y_true,y_pred,1.0)
def ssim(y_true,y_pred):
    return tf.image.ssim(y_true,y_pred,1.0)

In [3]:
input_size = 512 # 128x128x3 images (training_set)
batch_size = 1

## Load PreDefined Images

In [4]:
test_cloud_imgs = []
test_clear_imgs = []
test_cloud_dir = "C:/Users/ArrunPersonal/Codes/ISRO_GAN/PostComments_NewWeights/RICE2_TestImages/cloudy_image"
test_clear_dir = "C:/Users/ArrunPersonal/Codes/ISRO_GAN/PostComments_NewWeights/RICE2_TestImages/ground_truth"
names = []
for i in range(len(os.listdir(test_cloud_dir))):
    name = os.listdir(test_cloud_dir)[i]
    names.append(name)
    os.chdir(test_cloud_dir)
    test_cloud_imgs.append(np.array(Image.open(name))[:,:,:3]/255)
    os.chdir(test_clear_dir)
    test_clear_imgs.append(np.array(Image.open(name))[:,:,:3]/255) 

In [5]:
test_cloud_imgs = np.array(test_cloud_imgs)
test_clear_imgs = np.array(test_clear_imgs)

In [6]:
np.shape(test_clear_imgs),np.shape(test_cloud_imgs)

((14, 512, 512, 3), (14, 512, 512, 3))

## RS_CloudGAN Predictions 

In [7]:
def define_discriminator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_src_image = Input(shape=image_shape)
	# target image input
	in_target_image = Input(shape=image_shape)
	# concatenate images channel-wise
	merged = Concatenate()([in_src_image, in_target_image])
	# C64
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('sigmoid')(d)
	# define model
	model = Model([in_src_image, in_target_image], patch_out)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
	return model
 
# define image shape
image_shape = (input_size,input_size,3)
# create the model
model = define_discriminator(image_shape)
# summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 concatenate (Concatenate)      (None, 512, 512, 6)  0           ['input_1[0][0]',                
                                                                  'input_2[0][0]']            

C:\Users\ArrunPersonal\anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
C:\Users\ArrunPersonal\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [8]:
def encoder_block(input_features,num_filters, filter_size=[3,5,7]):
    conv1_1 = Conv2D(num_filters,filter_size[0], padding = 'same',activation = 'relu')(input_features)
    conv1_2 = Conv2D(num_filters,filter_size[1], padding = 'same',activation = 'relu')(input_features)
    conv1_3 = Conv2D(num_filters,filter_size[2], padding = 'same',activation = 'relu')(input_features)

    concat_12 = Concatenate()([conv1_1,conv1_2])
    concat_13 = Concatenate()([conv1_2,conv1_3])
    concat_23 = Concatenate()([conv1_1,conv1_3])

    conv2_1 = Conv2D(num_filters,3, padding = 'same',activation = 'relu')(concat_12)
    conv2_2 = Conv2D(num_filters,3, padding = 'same',activation = 'relu')(concat_23)
    conv2_3 = Conv2D(num_filters,3, padding = 'same',activation = 'relu')(concat_13)

    concat_123 = Concatenate()([conv2_1,conv2_2,conv2_3])
    conv_fin = Conv2D(num_filters,3, padding = 'same',activation = 'relu')(concat_123)
    maxpool_fin = MaxPool2D(2)(conv_fin)
    return maxpool_fin

In [9]:
def decoder_block(input_layer,skip_connection, num_filters,filter_size=[3,5,7]):
    convt1_1 = Conv2DTranspose(num_filters, filter_size[0], padding = 'same',activation = 'relu')(input_layer)
    convt1_2 = Conv2DTranspose(num_filters, filter_size[1], padding = 'same',activation = 'relu')(input_layer)
    convt1_3 = Conv2DTranspose(num_filters, filter_size[2], padding = 'same',activation = 'relu')(input_layer)

    concat_12 = Concatenate()([convt1_1,convt1_2])
    concat_13 = Concatenate()([convt1_2,convt1_3])
    concat_23 = Concatenate()([convt1_1,convt1_3])

    conv2_1 = Conv2D(num_filters,3, padding = 'same',activation = 'relu')(concat_12)
    conv2_2 = Conv2D(num_filters,3, padding = 'same',activation = 'relu')(concat_23)
    conv2_3 = Conv2D(num_filters,3, padding = 'same',activation = 'relu')(concat_13)

    concat_123 = Concatenate()([conv2_1,conv2_2,conv2_3,skip_connection])
    conv_fin = Conv2D(num_filters,3, padding = 'same',activation = 'relu')(concat_123)
    upsampling_fin = UpSampling2D(2)(conv_fin)
    
    return upsampling_fin

In [10]:
def bottleneck(input_layer,drop = 0.2):
    feature_layer = Conv2D(512,3,padding = 'same',activation = 'linear')(input_layer)
    attention_layer = Conv2D(512,3,padding = 'same',activation = 'sigmoid')(feature_layer)
    new_input_features = MultiHeadAttention(num_heads=3, key_dim=3, attention_axes=(2, 3))(input_layer,attention_layer)
    
    batch_norma = BatchNormalization()(new_input_features)
    if(drop):
        drop = Dropout(drop)(batch_norma)
        return drop
    return batch_norma

In [11]:
# def bottleneck(input_layer,drop = 0.1):
#     latent_img = Conv2D(3, 3, padding = 'same',activation = 'relu')(input_layer)
#     batch_norma = BatchNormalization()(latent_img)
#     if(drop):
#         drop = Dropout(drop)(batch_norma)
#         return drop
#     return batch_norma

## 3 layer

In [11]:
input_layer = Input((512,512,3))
e1 = encoder_block(input_layer,32)
e2 = encoder_block(e1,64)
e3 = encoder_block(e2,128)
b = bottleneck(e3)
d1 = decoder_block(b,e3,64)
d2 = decoder_block(d1,e2,32)
d3 = decoder_block(d2,e1,3)

CR_Net = Model(inputs = input_layer, outputs = d3)
CR_Net.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_6 (Conv2D)              (None, 512, 512, 32  4736        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_7 (Conv2D)              (None, 512, 512, 32  7808        ['input_3[0][0]']                
                                )                                                           

## 4 Layer

In [11]:
input_layer = Input((512,512,3))
e1 = encoder_block(input_layer,32)
e2 = encoder_block(e1,64)
e3 = encoder_block(e2,128)
e4 = encoder_block(e3,256)
b = bottleneck(e4)
d1 = decoder_block(b,e4,128)
d2 = decoder_block(d1,e3,64)
d3 = decoder_block(d2,e2,32)
d4 = decoder_block(d3,e1,3)

CR_Net = Model(inputs = input_layer, outputs = d4)
CR_Net.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_6 (Conv2D)              (None, 512, 512, 32  4736        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_7 (Conv2D)              (None, 512, 512, 32  7808        ['input_3[0][0]']                
                                )                                                           

## 5 Layer

In [11]:
input_layer = Input((512,512,3))
e1 = encoder_block(input_layer,32)
e2 = encoder_block(e1,64)
e3 = encoder_block(e2,128)
e4 = encoder_block(e3,256)
e5 = encoder_block(e4,512)
b = bottleneck(e5)
d1 = decoder_block(b,e5,256)
d2 = decoder_block(d1,e4,256)
d3 = decoder_block(d2,e3,128)
d4 = decoder_block(d3,e2,64)
d5 = decoder_block(d4,e1,3)

CR_Net = Model(inputs = input_layer, outputs = d5)
CR_Net.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_6 (Conv2D)              (None, 512, 512, 32  896         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_7 (Conv2D)              (None, 512, 512, 32  2432        ['input_3[0][0]']                
                                )                                                           

In [12]:
CR_Net.load_weights('C:/Users/ArrunPersonal/Codes/ISRO_GAN/PostComments_NewWeights/RICE2_weights/RS_CloudGAN_l5_k357_selfattn_bce_max_psnr_rice2.h5')

In [13]:
RS_Cloud_preds = CR_Net.predict(test_cloud_imgs,batch_size = 1)

14/14 [==============================] - 50s 539ms/step


## Net Preditions

In [14]:
def PSNR(GT, GAN_out):
    mse = np.mean((GT - GAN_out) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 10 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

In [15]:
from skimage.metrics import structural_similarity as ssim

In [16]:
PSNR_NewRS = []
SSIM_NewRS = []

for i in range(len(RS_Cloud_preds)):
    PSNR_NewRS.append(PSNR(test_clear_imgs[i],RS_Cloud_preds[i]))
    SSIM_NewRS.append(ssim(test_clear_imgs[i],RS_Cloud_preds[i],multichannel=True))

C:\Users\ArrunPersonal\anaconda3\lib\site-packages\skimage\metrics\_structural_similarity.py:107: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  ch_result = structural_similarity(im1[..., ch],


In [17]:
print(f"Avg. PSNR for New RS Predictions = {np.sum(PSNR_NewRS)/len(RS_Cloud_preds)}")

Avg. PSNR for New RS Predictions = 35.147545695764634


In [18]:
print(f"Avg. SSIM for New RS Predictions = {np.sum(SSIM_NewRS)/len(RS_Cloud_preds)}")

Avg. SSIM for New RS Predictions = 0.8750149743569321


In [17]:
print(f"Avg. PSNR for New RS Predictions = {np.sum(PSNR_NewRS)/len(RS_Cloud_preds)}")
print(f"Avg. SSIM for New RS Predictions = {np.sum(SSIM_NewRS)/len(RS_Cloud_preds)}")

Avg. PSNR for New RS Predictions = 36.0608289547362
Avg. SSIM for New RS Predictions = 0.8920808804762037
